In [7]:
import pandas as pd
import teradatasql
import datetime as dt

In [8]:
#Lee el archivo de los proveedores
def leer_archivos():
    
    #Especificar la ulr para agarrar todos los archivos de ahi, de momento no tengo la url de una compartida entonces por el momento no importa
    #url = ""
    nombre_cts = "Copia de CTS_Procesamiento_2025.06.16.xlsx"
    nombre_int = "Copia de Procesamiento_compras_INTENEG_2025-06-16.xlsx"
    nombre_historico = "Historico.xlsx"
    
    url_cts = nombre_cts
    url_int = nombre_int
    #url_his = url + nombre_historico
    
    dtype = {
        'Cuenta' : str,
        'Tarjeta' : str,
        'Autorizacion1' : str
    }

    #Lee los archivos tal cual como vienen en el sharepoint
    df_cts = pd.read_excel(url_cts, dtype=dtype, parse_dates=["Fcompra1"])
    df_int = pd.read_excel(url_int, dtype = dtype, parse_dates=["Fcompra1"])
    
    df_cts["Fcompra1"] = df_cts["Fcompra1"].dt.strftime("%d/%m/%Y")
    df_int["Fcompra1"] = df_int["Fcompra1"].dt.strftime("%d/%m/%Y")
    
    
    #Esto junta los dos df en uno solo pero no se si es necesario tenerlos separados o si si se puede tenerlos juntos
    #df_prove = pd.concat([df_cts,df_int])
    
    return df_cts, df_int

def quitar_duplicados(df):

    #Crea la llave
    df["Llave"] = df["Cuenta"] + df["Autorizacion1"]

    #Borramos los duplicados
    df = df.drop_duplicates(subset = 'Llave')
    
    return df
    
#def quitar_historicos(df):
    #Eliminamos las que ya se encuentrann en el historico
    #df_prove = df_prove[~df_prove['Llave'].isin(set(df_historico['Llave']))]

def formato_SQL(df,columna):
    
    #Convertir columnas en formato SQL (con comillas y coma) para la consulta SQL ejemplo 123456 va a ser igual a '123456',
    lista_formato_SQL = df[columna].tolist()
    lista_formato_SQL= ', '.join(f"'{tarjeta}'" for tarjeta in lista_formato_SQL)
    
    return lista_formato_SQL
    
    
def teradata_arsd(cuentas):
    teradata = f"""SELECT NUM_AUT_COM AS Compra,  NUM_CTA_CTAF AS Cuenta
    , (VAL_TRANS_COM - VAL_JUROS_PARC_COM) AS Monto
    , DAT_TRANS_COM AS fechacompra
    ,CASE WHEN VAL_DADO_ORIG_LKP IN ('320001','350001','353001','356001','310001','311001','373001','376001','372001','352001') THEN '1000'
    WHEN  VAL_DADO_ORIG_LKP  IN ('320002','350002','353002','356002','310002','311002','373002','376002','352002','372002','377002','357002','354002','374002') THEN '1100'
    WHEN  VAL_DADO_ORIG_LKP IN ('320003','350003','353003','356003','310003','311003','373003','376003','352003','372003','377003','357003','354003','374003') THEN '1181'
    END AS PLAN
    , NOM_ESTB_PRO
    ,'D002' AS Descripcion
    ,COD_CICL_FAT_FCCFM AS corte
    ,ID_PROD_CTAF
    ,Val_dado_orig_can 
    ,NUM_REF_COM as Reference 
    FROM COMPRA_V 
    INNER JOIN CARTAO_FINANCIAMENTO_V 
    ON ID_CAR_COM = ID_CAR_CARF 
    INNER JOIN CONTA_FINANCIAMENTO_V 
    ON ID_CTA_CTAF = ID_CTA_CARF 
    LEFT JOIN CANAL_V 
    ON ID_CANA_CAN=ID_CANA_ADES_CTAF 
    INNER JOIN TIPO_TRANSACAO_V 
    ON ID_TIPO_TRANS_LKP = ID_TIPO_TRANS_COM 
    INNER JOIN FAT_COMP_CONTA_FINANC_FDM_V 
    ON NUM_CTA_CTAF = NUM_CTA_ATUAL_FCCFM 
    AND ID_CICL_ATRA_ROLLRATE_FCCFM IN ('11','12') 
    AND VAL_SALDO_DIV_TOT_FCCFM > 0 
         INNER JOIN TRANSACAO_PROSA_V 
    ON ID_CAR_PRO=ID_CAR_COM 
    AND NUM_AUT_COM = NUM_AUT_PRO 
    AND NUM_REF_COM = NUM_REF_PRO 
    WHERE 
    ID_PROD_CTAF IN (9, 38, 24, 52, 11, 14, 19, 30, 36, 47,35,37,57,40)
    AND ((COD_CICL_FAT_FCCFM=10 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >10 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+10, -1)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+10,-1)  
     END AS z) AND CURRENT_DATE) 
     OR (COD_CICL_FAT_FCCFM=25 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >25 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+25, -1)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+25,-1)  
     END AS z) AND CURRENT_DATE)
      OR (COD_CICL_FAT_FCCFM=3 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >3 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+3, -1)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+3,-1)  
     END AS z) AND CURRENT_DATE)
                     OR (COD_CICL_FAT_FCCFM=5 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >5 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+5, -1)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+5,-1)  
     END AS z) AND CURRENT_DATE)
       OR (COD_CICL_FAT_FCCFM=15 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >15 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+15, -1)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+15,-1)  
     END AS z) AND CURRENT_DATE)
        OR (COD_CICL_FAT_FCCFM=20 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >20 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+20, -1)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+20,-1)  
     END AS z) AND CURRENT_DATE)
      OR (COD_CICL_FAT_FCCFM=11 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >11 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+11, -1)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+11,-1)  
     END AS z) AND CURRENT_DATE)
       OR (COD_CICL_FAT_FCCFM=28 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >28 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+28, -1)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+28,-1)  
     END AS z) AND CURRENT_DATE)
       OR (COD_CICL_FAT_FCCFM=23 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >23 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+23, -1)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+23,-1)  
     END AS z) AND CURRENT_DATE)
        OR (COD_CICL_FAT_FCCFM=8 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >8 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+8, -1)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+8,-1)  
     END AS z) AND CURRENT_DATE)
     )AND ID_MES_FCCFM = ADD_MONTHS(CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) +1, -1)
    AND VAL_DADO_ORIG_LKP IN ('320002', '320003', '320001','350002', '350003', '350001','353002', '353003', '353001','310002', '310003', '310001','311002', '311003', '311001','373002', '373003', '373001','376002', '376003', '376001','356002', '356003', '356001','352001','372001','352002','372002','352003','372003','377002','357002','354002','374002','377003','357003','354003','374003')
    AND NUM_CTA_CTAF IN ({cuentas})
    ORDER BY NUM_CTA_CTAF, Monto DESC
    """
    
    return teradata
    
def teradata_artd(cuentas):
    teradata = f"""SELECT NUM_AUT_COM AS Compra,  NUM_CTA_CTAF AS Cuenta
    , (VAL_TRANS_COM - VAL_JUROS_PARC_COM) AS Monto
    , DAT_TRANS_COM AS fechacompra
    ,CASE WHEN VAL_DADO_ORIG_LKP IN ('320001') THEN '1000'
    WHEN VAL_DADO_ORIG_LKP in ('311001','310001') THEN '6000'
    WHEN  VAL_DADO_ORIG_LKP  IN ('320002','350002','353002','356002','310002','311002','373002','376002','352002','372002','377002','357002','354002','374002') THEN '1100'
    WHEN  VAL_DADO_ORIG_LKP IN ('320003','350003','353003','356003','310003','311003','373003','376003','352003','372003','377003','357003','354003','374003') THEN '1181'
    END AS PLAN
    , NOM_ESTB_PRO
    ,'D002' AS Descripcion
    ,COD_CICL_FAT_FCCFM AS corte

    ,ID_PROD_CTAF

    ,Val_dado_orig_can 
    ,NUM_REF_COM as Reference 
    FROM COMPRA_V 
    INNER JOIN CARTAO_FINANCIAMENTO_V 
    ON ID_CAR_COM = ID_CAR_CARF 
    INNER JOIN CONTA_FINANCIAMENTO_V 
    ON ID_CTA_CTAF = ID_CTA_CARF 
    LEFT JOIN CANAL_V 
    ON ID_CANA_CAN=ID_CANA_ADES_CTAF 
    INNER JOIN TIPO_TRANSACAO_V 
    ON ID_TIPO_TRANS_LKP = ID_TIPO_TRANS_COM 
    INNER JOIN FAT_COMP_CONTA_FINANC_FDM_V 
    ON NUM_CTA_CTAF = NUM_CTA_ATUAL_FCCFM 
    AND ID_CICL_ATRA_ROLLRATE_FCCFM IN ('11','12') 
    AND VAL_SALDO_DIV_TOT_FCCFM > 0 
         full OUTER JOIN TRANSACAO_PROSA_V 
    ON ID_CAR_PRO=ID_CAR_COM 
    AND NUM_AUT_COM = NUM_AUT_PRO 
    AND NUM_REF_COM = NUM_REF_PRO 
    WHERE 
    ID_PROD_CTAF IN (9, 38, 24, 52, 11, 14, 19, 30, 36, 47,35,37,57,40)
    AND ((COD_CICL_FAT_FCCFM=10 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >10 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+10, +0)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+10,-1)  
     END AS z) AND CURRENT_DATE) 
     OR (COD_CICL_FAT_FCCFM=25 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >25 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+25, +0)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+25,-1)  
     END AS z) AND CURRENT_DATE)
      OR (COD_CICL_FAT_FCCFM=3 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >3 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+3, +0)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+3,-1)  
     END AS z) AND CURRENT_DATE)
      OR (COD_CICL_FAT_FCCFM=5 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >5 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+5, +0)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+5,-1)  
     END AS z) AND CURRENT_DATE)
       OR (COD_CICL_FAT_FCCFM=15 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >15 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+15, +0)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+15,-1)  
     END AS z) AND CURRENT_DATE)
        OR (COD_CICL_FAT_FCCFM=20 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >20 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+20, +0)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+20,-1)  
     END AS z) AND CURRENT_DATE)
         OR (COD_CICL_FAT_FCCFM=28 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >28 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+28, +0)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+28,-1)  
     END AS z) AND CURRENT_DATE)
         OR (COD_CICL_FAT_FCCFM=23 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >23 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+23, +0)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+23,-1)  
     END AS z) AND CURRENT_DATE)
         OR (COD_CICL_FAT_FCCFM=8 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >8 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+8, +0)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+8,-1)  
     END AS z) AND CURRENT_DATE)
      OR (COD_CICL_FAT_FCCFM=11 AND DAT_TRANS_COM BETWEEN(SELECT CASE  WHEN  EXTRACT(DAY FROM CURRENT_DATE) >11 THEN  ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+11, +0)  
    ELSE ADD_MONTHS((CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) )+11,-1)  
     END AS z) AND CURRENT_DATE)
     )AND (ID_MES_FCCFM = ADD_MONTHS(CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) +1, -1)
     or ID_MES_FCCFM = ADD_MONTHS(CURRENT_DATE- EXTRACT(DAY FROM CURRENT_DATE) +1, 0))
    AND VAL_DADO_ORIG_LKP IN ('320002', '320003', '320001','350002', '350003', '350001','353002', '353003', '353001','310002', '310003', '310001','311002', '311003', '311001','373002', '373003', '373001','376002', '376003', '376001','356002', '356003', '356001','352001','372001','352002','372002','352003','372003','377002','357002','354002','374002','377003','357003','354003','374003')
    AND NUM_CTA_CTAF IN ({cuentas})
    ORDER BY NUM_CTA_CTAF, Monto DESC
    
    """
    
    return teradata

def conexion_a_teradata():
    conn = teradatasql.connect(
        host='192.168.17.64',
        user="MPROC_GR", 
        password="padrao",
        database="PM_MSTRDB",


    )
    
def hacer_consulta(query):
    df = pd.read_sql(query, conn)
    
    return df

def cerrar_conexion(query):
    conn.close()

    
def main():
    
    
    #Para no hacer todo dos veces en caso de qu los df se deban mantener separados, se puede crear una funcion en donde se ponga 
    #todos los pasos y de parametro reciba un df, finalmente en el main llamamos a esa funcio dos veces una por cada df
    df_cts, df_int = leer_archivos()
    df_cts = quitar_duplicados(df_cts)
    df_int = quitar_duplicados(df_int)
    
    #Convierte todas las cuentas en formato SQL
    cuentas_cts = formato_SQL(df_cts, "Cuenta")
    cuentas_int = formato_SQL(df_int, "Cuenta")
    cuentas = cuentas_cts + ',' +cuentas_int
    
    #Hace la consulta en teradata
    #conexion_a_teradata()
    #df_arsd = hacer_consulta(teradata_arsd(cuentas))
    #df_artd = hacer_consulta(teradata_artd(cuentas))
    #cerrar_conexion()
    #NOSE SI FUNCIONA NECESITO LOS DATOS DE TERADATA PARA INTENARLO, DE MIENTRAS PUSE LAS CONSULTAS DE TERADATA EN UN EXCEL
    
    #Hacer cruces con la consulta para ver si estan los casos ahi
    #La consulta las tengo de momento en excel
    
    dtype = {
        'Compra' : str,
        'Cuenta' : str,
        'Reference': str
            }
    
    df_arsd = pd.read_excel("arsd.xlsx", dtype=dtype)
    df_artd = pd.read_excel("artd.xlsx", dtype = dtype)
    
    #Concatenar las dos consultas
    df_teradata = pd.concat([df_arsd,df_artd])
    
    #Crear una llave en terada cuenta&aut&monto&fecha?
    df_teradata["Llave"] = df_teradata["Cuenta"] + df_teradata["Compra"] + df_teradata["Monto"].astype(str) + df_teradata["fechacompra"].astype(str)
    df_cts["Llave_completa"] = df_cts["Cuenta"] + df_cts["Autorizacion1"] + df_cts["Monto1"].astype(str) + df_cts["Fcompra1"].astype(str)
    
    #Busca en las dos consultas de teradata si se encuentran esos casos y asi saber que todos los campos de la llave estan bien
    df_cts = df_cts[df_cts['Llave_completa'].isin(set(df_teradata['Llave']))]
    
    #Los que no tienen algun campo bien, averiguar que campo esta mal y poner el correcto?
    
    
    print(df_cts)
    
    
    
main()



                   Cuenta              Tarjeta  \
0     0004037500115133000     4037500115133017   
1     0004037500165058000     4037500214042879   
2     0004037500216466068     4037500216466068   
3     0004037500216594281     4037500216594281   
4     0004463511523384000     4463511523384020   
...                   ...                  ...   
7487  0004812840200558000  0004812840200558053   
7488  0004812840200558000  0004812840200558053   
7489  0004812840595640000  0004812840595640045   
7490  0004812840595640000  0004812840595640045   
7491  0004812840756002000  0004812840795822278   

                               Nombre Autorizacion1   Monto1    Fcompra1  \
0            HERNANDEZ ACOSTA  FELIPE        003937  1709.50  01/06/2025   
1       VALDERRAMA RAMIREZ  FRANCISCO        021606   344.85  02/06/2025   
2     CISNEROS PERALES  LINDA CECILIA        095596   828.00  12/06/2025   
3      VAZQUEZ HERNANDEZ  JUAN CARLOS        074258  1200.00  14/06/2025   
4                RA